# Custom Field Updates - Comprehensive Data Types

This notebook provides comprehensive examples of updating custom fields on accounts with various data types.
It covers text, numeric, boolean, date, list, and complex data type updates.

**Purpose**: Complete custom field update examples with all data types
**Target**: SDK development and custom field implementation

**⚠️ WARNING: This notebook contains UPDATE operations that will modify data.**
**Uncomment and modify the examples as needed for your testing environment.**

In [1]:
# Import required libraries
import os
import sys
import json
import time
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Union
from decimal import Decimal
import warnings

# Add the src directory to the path
sys.path.insert(0, "../src")

from neon_crm import NeonClient
from neon_crm.exceptions import NeonAPIError, NeonValidationError

# Configure logging for detailed output
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

print("✅ All imports successful")
print(f"📅 Custom field updates testing started at: {datetime.now()}")

✅ All imports successful
📅 Custom field updates testing started at: 2025-10-04 15:27:57.491710


In [2]:
# Initialize the clienttry:    client = NeonClient()    print("✅ Client initialized successfully")    # Test connectivity    test_response = client.accounts.get_search_fields()    print(f"✅ API connectivity confirmed")except Exception as e:    print(f"❌ Failed to initialize client: {str(e)}")    print("Please ensure NEON_ORG_ID and NEON_API_KEY environment variables are set")    raise# Initialize test_account_id placeholdertest_account_id = None

## Section 1: Discover Available Custom Fields

First, let's discover what custom fields are available for accounts in your organization.

In [3]:
# 1.1: Discover custom fields for accounts
print("🔍 SECTION 1.1: Discovering Custom Fields for Accounts")
print("=" * 55)


def discover_account_custom_fields(client):
    """Discover all custom fields available for accounts."""

    custom_fields_info = {
        "search_custom_fields": [],
        "output_custom_fields": [],
        "likely_custom_patterns": [],
        "discovered_timestamp": datetime.now().isoformat(),
    }

    try:
        # Get search fields
        search_fields_response = client.accounts.get_search_fields()

        # Extract declared custom fields
        custom_search = search_fields_response.get("customFields", [])
        if custom_search:
            if isinstance(custom_search, list) and len(custom_search) > 0:
                if isinstance(custom_search[0], dict):
                    custom_fields_info["search_custom_fields"] = [
                        {
                            "fieldName": f.get("fieldName", str(f)),
                            "fieldType": f.get("fieldType", "unknown"),
                            "required": f.get("required", False),
                        }
                        for f in custom_search
                    ]
                else:
                    custom_fields_info["search_custom_fields"] = [
                        {"fieldName": str(f), "fieldType": "unknown", "required": False}
                        for f in custom_search
                    ]

        # Check standard fields for custom-like patterns
        standard_search = [
            f["fieldName"] for f in search_fields_response.get("standardFields", [])
        ]

        # Look for custom patterns in standard fields
        import re

        custom_patterns = {
            "numeric_id": re.compile(r"^\d+$"),
            "prefixed_id": re.compile(r"^[A-Z]-\d+$"),
            "dash_separated": re.compile(r"^.+\s-\s.+$"),
            "custom_prefix": re.compile(r"^(Custom|C-).*", re.IGNORECASE),
            "very_long": lambda x: len(x) > 50,
        }

        for field_name in standard_search:
            for pattern_name, pattern in custom_patterns.items():
                if pattern_name == "very_long":
                    if pattern(field_name):
                        custom_fields_info["likely_custom_patterns"].append(
                            {
                                "fieldName": field_name,
                                "pattern": pattern_name,
                                "fieldType": "inferred_text",
                            }
                        )
                        break
                elif hasattr(pattern, "match") and pattern.match(field_name):
                    # Infer type based on pattern
                    inferred_type = "text"
                    if pattern_name == "numeric_id":
                        inferred_type = "number"
                    elif "date" in field_name.lower():
                        inferred_type = "date"
                    elif any(
                        word in field_name.lower()
                        for word in ["active", "enabled", "flag", "is_"]
                    ):
                        inferred_type = "boolean"

                    custom_fields_info["likely_custom_patterns"].append(
                        {
                            "fieldName": field_name,
                            "pattern": pattern_name,
                            "fieldType": f"inferred_{inferred_type}",
                        }
                    )
                    break

        print(f"✅ Custom field discovery completed")

    except Exception as e:
        print(f"❌ Error during custom field discovery: {str(e)}")
        custom_fields_info["error"] = str(e)

    try:
        # Get output fields
        output_fields_response = client.accounts.get_output_fields()

        # Extract declared custom output fields
        custom_output = output_fields_response.get("customFields", [])
        if custom_output:
            if isinstance(custom_output, list) and len(custom_output) > 0:
                if isinstance(custom_output[0], dict):
                    custom_fields_info["output_custom_fields"] = [
                        {
                            "fieldName": f.get("fieldName", str(f)),
                            "fieldType": f.get("fieldType", "unknown"),
                        }
                        for f in custom_output
                    ]
                else:
                    custom_fields_info["output_custom_fields"] = [
                        {"fieldName": str(f), "fieldType": "unknown"}
                        for f in custom_output
                    ]

    except Exception as e:
        print(f"⚠️  Error getting output fields: {str(e)}")

    return custom_fields_info


# Discover custom fields
custom_fields_info = discover_account_custom_fields(client)

# Display discovery results
print("\n📊 CUSTOM FIELDS DISCOVERY RESULTS:")
print("=" * 40)

declared_search = custom_fields_info.get("search_custom_fields", [])
declared_output = custom_fields_info.get("output_custom_fields", [])
likely_custom = custom_fields_info.get("likely_custom_patterns", [])

print(f"Declared custom search fields: {len(declared_search)}")
if declared_search:
    for field in declared_search[:5]:  # Show first 5
        print(f"  - {field['fieldName']} ({field['fieldType']})")
    if len(declared_search) > 5:
        print(f"  ... and {len(declared_search) - 5} more")

print(f"\nDeclared custom output fields: {len(declared_output)}")
if declared_output:
    for field in declared_output[:5]:  # Show first 5
        print(f"  - {field['fieldName']} ({field['fieldType']})")
    if len(declared_output) > 5:
        print(f"  ... and {len(declared_output) - 5} more")

print(f"\nLikely custom patterns in standard fields: {len(likely_custom)}")
if likely_custom:
    for field in likely_custom[:5]:  # Show first 5
        print(
            f"  - {field['fieldName']} (pattern: {field['pattern']}, type: {field['fieldType']})"
        )
    if len(likely_custom) > 5:
        print(f"  ... and {len(likely_custom) - 5} more")

# Combine all discovered custom fields for testing
all_custom_fields = []
all_custom_fields.extend(declared_search)
all_custom_fields.extend(declared_output)
all_custom_fields.extend(likely_custom)

# Remove duplicates based on field name
unique_custom_fields = {}
for field in all_custom_fields:
    field_name = field["fieldName"]
    if field_name not in unique_custom_fields:
        unique_custom_fields[field_name] = field

all_custom_fields = list(unique_custom_fields.values())

print(f"\n🎯 Total unique custom fields discovered: {len(all_custom_fields)}")

if not all_custom_fields:
    print(
        "\n⚠️  No custom fields discovered. We'll use example field names for demonstration."
    )
    # Create example custom fields for demonstration
    all_custom_fields = [
        {"fieldName": "123", "fieldType": "text"},
        {"fieldName": "456", "fieldType": "number"},
        {"fieldName": "Custom - Preferred Contact Method", "fieldType": "text"},
        {"fieldName": "Custom - Is VIP", "fieldType": "boolean"},
        {"fieldName": "Custom - Last Event Date", "fieldType": "date"},
    ]
    print("  📝 Using example custom fields for demonstration purposes")

🔍 SECTION 1.1: Discovering Custom Fields for Accounts


NameError: name 'client' is not defined

## Section 2: Get Sample Account for Testing

Find a sample account to use for custom field update testing.

In [ ]:
# Get a sample account for testingprint("🔍 Finding sample account for testing...")search_request = {    "searchFields": [],    "outputFields": ["Account ID", "First Name", "Last Name"],    "pagination": {"currentPage": 0, "pageSize": 1}}try:    accounts = list(client.accounts.search(search_request))    if accounts:        test_account_id = str(accounts[0].get('Account ID', ''))        print(f"✅ Found test account: {test_account_id}")    else:        test_account_id = "12345"        print("⚠️  No accounts found, using placeholder")except Exception as e:    test_account_id = "12345"    print(f"⚠️  Error: {e}, using placeholder")print(f"📋 test_account_id = {test_account_id}")

## Section 3: Custom Field Update Examples by Data Type

Comprehensive examples of updating custom fields with different data types.

**⚠️ WARNING: The following examples contain UPDATE operations that will modify data.**
**They are commented out by default. Uncomment and modify as needed for your testing.**

In [ ]:
# 3.1: Text/String Custom Field Updates
print("🔍 SECTION 3.1: Text/String Custom Field Updates")
print("=" * 50)


def update_text_custom_field_example(
    client, account_id: str, field_name: str, new_value: str
):
    """Example of updating a text custom field."""

    print(f"\n📝 Updating text custom field '{field_name}' to '{new_value}'")

    # Prepare update data
    update_data = {field_name: new_value}

    print(f"\n🔧 Update data prepared:")
    print(f"  Field: {field_name}")
    print(f"  Value: {new_value}")
    print(f"  Type: {type(new_value).__name__}")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update text custom field")
    print(f"update_data = {{")
    print(f"    '{field_name}': '{new_value}'")
    print(f"}}")
    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Text custom field updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating text custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, update_data)
    #     print(f"✅ Text custom field '{field_name}' updated successfully")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating text custom field: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example text custom field updates
text_examples = [
    {
        "field_name": "Custom - Preferred Contact Method",
        "value": "Email 1",
        "description": "Simple text value",
    },
    {
        "field_name": "Custom - Notes",
        "value": "High-value donor with specific interests in environmental causes.",
        "description": "Long text with description",
    },
    {
        "field_name": "Custom - Department",
        "value": "Marketing & Communications",
        "description": "Text with special characters",
    },
    {
        "field_name": "123",  # Numeric field name
        "value": "Custom Value for Numeric Field ID",
        "description": "Text value in numeric field name",
    },
]

print("\n📝 TEXT CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 40)

for i, example in enumerate(text_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        update_text_custom_field_example(
            client, test_account_id, example["field_name"], example["value"]
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']}")
        print(f"   📝 No test account available - showing structure only")

In [ ]:
# 3.2: Numeric Custom Field Updates
print("\n🔍 SECTION 3.2: Numeric Custom Field Updates")
print("=" * 50)


def update_numeric_custom_field_example(
    client, account_id: str, field_name: str, new_value: Union[int, float, str]
):
    """Example of updating a numeric custom field."""

    print(f"\n🔢 Updating numeric custom field '{field_name}' to {new_value}")

    # Prepare update data
    update_data = {field_name: new_value}

    print(f"\n🔧 Update data prepared:")
    print(f"  Field: {field_name}")
    print(f"  Value: {new_value}")
    print(f"  Type: {type(new_value).__name__}")
    print(f"  String representation: '{str(new_value)}'")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update numeric custom field")
    print(f"update_data = {{")
    if isinstance(new_value, str):
        print(f"    '{field_name}': '{new_value}'  # String representation of number")
    else:
        print(f"    '{field_name}': {new_value}  # Numeric value")
    print(f"}}")
    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Numeric custom field updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating numeric custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, update_data)
    #     print(f"✅ Numeric custom field '{field_name}' updated successfully")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating numeric custom field: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example numeric custom field updates
numeric_examples = [
    {
        "field_name": "Custom - Donation Count",
        "value": 15,
        "description": "Integer value",
    },
    {
        "field_name": "Custom - Average Gift Amount",
        "value": 125.50,
        "description": "Float value with decimals",
    },
    {
        "field_name": "Custom - Total Lifetime Value",
        "value": "1250.75",
        "description": "Numeric value as string",
    },
    {
        "field_name": "456",  # Numeric field name
        "value": 42,
        "description": "Integer in numeric field ID",
    },
    {"field_name": "Custom - Score", "value": 0, "description": "Zero value"},
    {"field_name": "Custom - Rating", "value": -1, "description": "Negative value"},
]

print("\n🔢 NUMERIC CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 45)

for i, example in enumerate(numeric_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        update_numeric_custom_field_example(
            client, test_account_id, example["field_name"], example["value"]
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']} ({type(example['value']).__name__})")
        print(f"   📝 No test account available - showing structure only")

In [ ]:
# 3.3: Boolean Custom Field Updates
print("\n🔍 SECTION 3.3: Boolean Custom Field Updates")
print("=" * 50)


def update_boolean_custom_field_example(
    client, account_id: str, field_name: str, new_value: Union[bool, str]
):
    """Example of updating a boolean custom field."""

    print(f"\n✅ Updating boolean custom field '{field_name}' to {new_value}")

    # Prepare update data
    update_data = {field_name: new_value}

    print(f"\n🔧 Update data prepared:")
    print(f"  Field: {field_name}")
    print(f"  Value: {new_value}")
    print(f"  Type: {type(new_value).__name__}")
    print(f"  Boolean interpretation: {bool(new_value)}")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update boolean custom field")
    print(f"update_data = {{")
    if isinstance(new_value, str):
        print(f"    '{field_name}': '{new_value}'  # String representation")
    else:
        print(f"    '{field_name}': {new_value}  # Boolean value")
    print(f"}}")
    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Boolean custom field updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating boolean custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, update_data)
    #     print(f"✅ Boolean custom field '{field_name}' updated successfully")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating boolean custom field: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example boolean custom field updates
boolean_examples = [
    {
        "field_name": "Custom - Is VIP",
        "value": True,
        "description": "Boolean True value",
    },
    {
        "field_name": "Custom - Email Opt Out",
        "value": False,
        "description": "Boolean False value",
    },
    {
        "field_name": "Custom - Active Member",
        "value": "true",
        "description": 'String "true" value',
    },
    {
        "field_name": "Custom - Volunteer",
        "value": "false",
        "description": 'String "false" value',
    },
    {
        "field_name": "Custom - Newsletter Subscriber",
        "value": "yes",
        "description": 'String "yes" value',
    },
    {
        "field_name": "Custom - Phone Consent",
        "value": "no",
        "description": 'String "no" value',
    },
    {
        "field_name": "Custom - Board Member",
        "value": 1,
        "description": "Numeric 1 (truthy)",
    },
    {
        "field_name": "Custom - Former Donor",
        "value": 0,
        "description": "Numeric 0 (falsy)",
    },
]

print("\n✅ BOOLEAN CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 45)

for i, example in enumerate(boolean_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        update_boolean_custom_field_example(
            client, test_account_id, example["field_name"], example["value"]
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']} ({type(example['value']).__name__})")
        print(f"   Boolean interpretation: {bool(example['value'])}")
        print(f"   📝 No test account available - showing structure only")

In [ ]:
# 3.4: Date Custom Field Updates
print("\n🔍 SECTION 3.4: Date Custom Field Updates")
print("=" * 50)


def update_date_custom_field_example(
    client, account_id: str, field_name: str, new_value: str
):
    """Example of updating a date custom field."""

    print(f"\n📅 Updating date custom field '{field_name}' to {new_value}")

    # Prepare update data
    update_data = {field_name: new_value}

    print(f"\n🔧 Update data prepared:")
    print(f"  Field: {field_name}")
    print(f"  Value: {new_value}")
    print(f"  Type: {type(new_value).__name__}")

    # Validate date format
    try:
        from datetime import datetime

        parsed_date = datetime.strptime(new_value, "%Y-%m-%d")
        print(f"  Parsed date: {parsed_date.strftime('%B %d, %Y')}")
        print(f"  ✅ Valid date format")
    except ValueError:
        print(f"  ⚠️  Date format may not be standard YYYY-MM-DD")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update date custom field (use YYYY-MM-DD format)")
    print(f"from datetime import datetime")
    print(f"")
    print(f"# Ensure proper date format")
    print(f"date_value = '{new_value}'  # YYYY-MM-DD format")
    print(f"")
    print(f"update_data = {{")
    print(f"    '{field_name}': date_value")
    print(f"}}")
    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Date custom field updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating date custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, update_data)
    #     print(f"✅ Date custom field '{field_name}' updated successfully")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating date custom field: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Generate some date examples
today = datetime.now()
yesterday = today - timedelta(days=1)
next_week = today + timedelta(days=7)
last_year = today.replace(year=today.year - 1)

# Example date custom field updates
date_examples = [
    {
        "field_name": "Custom - Last Contact Date",
        "value": today.strftime("%Y-%m-%d"),
        "description": "Today's date",
    },
    {
        "field_name": "Custom - Last Event Date",
        "value": yesterday.strftime("%Y-%m-%d"),
        "description": "Yesterday's date",
    },
    {
        "field_name": "Custom - Next Follow Up",
        "value": next_week.strftime("%Y-%m-%d"),
        "description": "Future date (next week)",
    },
    {
        "field_name": "Custom - Member Since",
        "value": last_year.strftime("%Y-%m-%d"),
        "description": "Historical date (last year)",
    },
    {
        "field_name": "Custom - Birth Date",
        "value": "1990-05-15",
        "description": "Specific birth date",
    },
    {
        "field_name": "Custom - Anniversary",
        "value": "2020-12-25",
        "description": "Holiday date",
    },
]

print("\n📅 DATE CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 40)

for i, example in enumerate(date_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        update_date_custom_field_example(
            client, test_account_id, example["field_name"], example["value"]
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']}")
        print(f"   📝 No test account available - showing structure only")

In [ ]:
# 3.5: List/Array Custom Field Updates
print("\n🔍 SECTION 3.5: List/Array Custom Field Updates")
print("=" * 50)


def update_list_custom_field_example(
    client, account_id: str, field_name: str, new_value: Union[List, str]
):
    """Example of updating a list/array custom field."""

    print(f"\n📋 Updating list custom field '{field_name}' with {new_value}")

    # Prepare update data
    update_data = {field_name: new_value}

    print(f"\n🔧 Update data prepared:")
    print(f"  Field: {field_name}")
    print(f"  Value: {new_value}")
    print(f"  Type: {type(new_value).__name__}")

    if isinstance(new_value, list):
        print(f"  List length: {len(new_value)}")
        print(f"  Items: {', '.join(str(item) for item in new_value)}")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update list custom field")
    if isinstance(new_value, list):
        print(f"# Option 1: Pass as Python list")
        print(f"update_data = {{")
        print(f"    '{field_name}': {new_value}")
        print(f"}}")
        print(f"")
        print(f"# Option 2: Pass as JSON string")
        import json

        json_string = json.dumps(new_value)
        print(f"update_data = {{")
        print(f"    '{field_name}': '{json_string}'")
        print(f"}}")
    else:
        print(f"# Pass as string (comma-separated or JSON)")
        print(f"update_data = {{")
        print(f"    '{field_name}': '{new_value}'")
        print(f"}}")

    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ List custom field updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating list custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, update_data)
    #     print(f"✅ List custom field '{field_name}' updated successfully")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating list custom field: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example list custom field updates
list_examples = [
    {
        "field_name": "Custom - Interests",
        "value": ["Environment", "Education", "Healthcare"],
        "description": "List of strings",
    },
    {
        "field_name": "Custom - Communication Preferences",
        "value": ["Email 1", "Phone", "Mail"],
        "description": "Multiple communication methods",
    },
    {
        "field_name": "Custom - Skills",
        "value": "Marketing, Finance, Event Planning",
        "description": "Comma-separated string",
    },
    {
        "field_name": "Custom - Tags",
        "value": '["VIP", "Board Member", "Volunteer"]',
        "description": "JSON array string",
    },
    {
        "field_name": "Custom - Event History",
        "value": ["2023 Gala", "2023 Golf Tournament", "2024 5K Run"],
        "description": "Event attendance list",
    },
    {
        "field_name": "Custom - Volunteer Roles",
        "value": [],
        "description": "Empty list",
    },
]

print("\n📋 LIST/ARRAY CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 50)

for i, example in enumerate(list_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        update_list_custom_field_example(
            client, test_account_id, example["field_name"], example["value"]
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']} ({type(example['value']).__name__})")
        print(f"   📝 No test account available - showing structure only")

In [ ]:
# 3.6: Complex/Object Custom Field Updates
print("\n🔍 SECTION 3.6: Complex/Object Custom Field Updates")
print("=" * 55)


def update_complex_custom_field_example(
    client, account_id: str, field_name: str, new_value: Union[Dict, str]
):
    """Example of updating a complex/object custom field."""

    print(f"\n🏗️  Updating complex custom field '{field_name}' with structured data")

    # Prepare update data
    update_data = {field_name: new_value}

    print(f"\n🔧 Update data prepared:")
    print(f"  Field: {field_name}")
    print(f"  Value: {new_value}")
    print(f"  Type: {type(new_value).__name__}")

    if isinstance(new_value, dict):
        print(f"  Object keys: {list(new_value.keys())}")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update complex custom field")
    if isinstance(new_value, dict):
        print(f"# Option 1: Pass as Python dictionary")
        print(f"complex_data = {{")
        for key, value in new_value.items():
            if isinstance(value, str):
                print(f"    '{key}': '{value}',")
            else:
                print(f"    '{key}': {value},")
        print(f"}}")
        print(f"")
        print(f"update_data = {{")
        print(f"    '{field_name}': complex_data")
        print(f"}}")
        print(f"")
        print(f"# Option 2: Pass as JSON string")
        import json

        json_string = json.dumps(new_value)
        print(f"update_data = {{")
        print(f"    '{field_name}': '{json_string}'")
        print(f"}}")
    else:
        print(f"# Pass as JSON string")
        print(f"update_data = {{")
        print(f"    '{field_name}': '{new_value}'")
        print(f"}}")

    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Complex custom field updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating complex custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, update_data)
    #     print(f"✅ Complex custom field '{field_name}' updated successfully")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating complex custom field: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example complex custom field updates
complex_examples = [
    {
        "field_name": "Custom - Preferences",
        "value": {
            "email_frequency": "weekly",
            "communication_type": "newsletter",
            "language": "English",
            "time_zone": "EST",
        },
        "description": "Nested preferences object",
    },
    {
        "field_name": "Custom - Address Details",
        "value": {
            "type": "residential",
            "verified": True,
            "last_updated": "2024-01-15",
            "source": "user_provided",
        },
        "description": "Address metadata object",
    },
    {
        "field_name": "Custom - Social Media",
        "value": {
            "platforms": ["Twitter", "LinkedIn", "Facebook"],
            "primary": "LinkedIn",
            "public_profile": True,
        },
        "description": "Social media information",
    },
    {
        "field_name": "Custom - Emergency Contact",
        "value": '{"name": "John Doe", "relationship": "spouse", "phone": "555-0123", "email": "john@example.com"}',
        "description": "JSON string with contact info",
    },
    {
        "field_name": "Custom - Donation History Summary",
        "value": {
            "total_amount": 1250.00,
            "donation_count": 15,
            "first_donation": "2020-03-15",
            "last_donation": "2024-01-10",
            "average_gift": 83.33,
            "largest_gift": 500.00,
        },
        "description": "Donation statistics object",
    },
]

print("\n🏗️  COMPLEX/OBJECT CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 55)

for i, example in enumerate(complex_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        update_complex_custom_field_example(
            client, test_account_id, example["field_name"], example["value"]
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']}")
        print(f"   Type: {type(example['value']).__name__}")
        print(f"   📝 No test account available - showing structure only")

## Section 4: Multiple Custom Field Updates

Examples of updating multiple custom fields in a single operation.

In [ ]:
# 4.1: Multiple custom field updates in single operation
print("🔍 SECTION 4.1: Multiple Custom Field Updates")
print("=" * 50)


def update_multiple_custom_fields_example(
    client, account_id: str, field_updates: Dict[str, Any]
):
    """Example of updating multiple custom fields in a single operation."""

    print(f"\n🔄 Updating {len(field_updates)} custom fields simultaneously")

    print(f"\n🔧 Update data prepared:")
    for field_name, value in field_updates.items():
        print(f"  {field_name}: {value} ({type(value).__name__})")

    print(f"\n💡 Code example:")
    print(f"```python")
    print(f"# Update multiple custom fields at once")
    print(f"update_data = {{")
    for field_name, value in field_updates.items():
        if isinstance(value, str):
            print(f"    '{field_name}': '{value}',")
        elif isinstance(value, (list, dict)):
            print(f"    '{field_name}': {value},")
        else:
            print(f"    '{field_name}': {value},")
    print(f"}}")
    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Multiple custom fields updated successfully')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating multiple custom fields: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, field_updates)
    #     print(f"✅ Multiple custom fields updated successfully")
    #     print(f"   Updated fields: {', '.join(field_updates.keys())}")
    #     return result
    # except Exception as e:
    #     print(f"❌ Error updating multiple custom fields: {str(e)}")
    #     return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example multiple custom field updates
multiple_field_examples = [
    {
        "name": "Complete Profile Update",
        "fields": {
            "Custom - Preferred Contact Method": "Email 1",
            "Custom - Is VIP": True,
            "Custom - Last Contact Date": datetime.now().strftime("%Y-%m-%d"),
            "Custom - Donation Count": 5,
            "Custom - Interests": ["Environment", "Education"],
            "Custom - Newsletter Subscriber": "yes",
        },
    },
    {
        "name": "Communication Preferences",
        "fields": {
            "Custom - Email Opt Out": False,
            "Custom - Phone Consent": True,
            "Custom - Communication Preferences": ["Email 1", "Phone"],
            "Custom - Language": "English",
            "Custom - Time Zone": "EST",
        },
    },
    {
        "name": "Engagement Tracking",
        "fields": {
            "Custom - Volunteer": True,
            "Custom - Board Member": False,
            "Custom - Event History": ["2023 Gala", "2024 5K Run"],
            "Custom - Skills": "Marketing, Finance",
            "Custom - Availability": "Weekends",
        },
    },
    {
        "name": "Mixed Data Types",
        "fields": {
            "123": "Numeric field with text",
            "456": 42,
            "Custom - Active Member": True,
            "Custom - Member Since": "2020-01-15",
            "Custom - Preferences": {"email_frequency": "monthly", "format": "html"},
        },
    },
]

print("\n🔄 MULTIPLE CUSTOM FIELD UPDATE EXAMPLES:")
print("=" * 50)

for i, example in enumerate(multiple_field_examples, 1):
    print(f"\n{i}. {example['name']}:")
    print(f"   Fields to update: {len(example['fields'])}")

    if test_account_id:
        update_multiple_custom_fields_example(
            client, test_account_id, example["fields"]
        )
    else:
        print(f"   📝 No test account available - showing structure only")
        for field_name, value in example["fields"].items():
            print(f"     {field_name}: {value} ({type(value).__name__})")

## Section 5: Advanced Custom Field Operations

Advanced patterns for custom field management including validation, error handling, and best practices.

In [ ]:
# 5.1: Custom field validation before update
print("🔍 SECTION 5.1: Custom Field Validation Before Update")
print("=" * 55)


def validate_and_update_custom_field(
    client, account_id: str, field_name: str, value: Any, field_type: str = None
):
    """Advanced example with validation before updating custom fields."""

    print(f"\n🔍 Validating and updating custom field '{field_name}'")

    validation_errors = []
    warnings = []

    # Step 1: Validate field name pattern
    import re

    custom_patterns = {
        "numeric_id": re.compile(r"^\d+$"),
        "prefixed_id": re.compile(r"^[A-Z]-\d+$"),
        "dash_separated": re.compile(r"^.+\s-\s.+$"),
        "custom_prefix": re.compile(r"^(Custom|C-).*", re.IGNORECASE),
    }

    field_pattern_valid = False
    for pattern_name, pattern in custom_patterns.items():
        if pattern.match(field_name):
            field_pattern_valid = True
            print(f"   ✅ Field name matches pattern: {pattern_name}")
            break

    if not field_pattern_valid:
        warnings.append(
            f"Field name '{field_name}' doesn't match known custom field patterns"
        )

    # Step 2: Validate value based on inferred or specified type
    if field_type == "date" or "date" in field_name.lower():
        # Validate date format
        try:
            from datetime import datetime

            if isinstance(value, str):
                datetime.strptime(value, "%Y-%m-%d")
                print(f"   ✅ Valid date format: {value}")
            else:
                validation_errors.append(
                    f"Date field requires string in YYYY-MM-DD format, got {type(value).__name__}"
                )
        except ValueError:
            validation_errors.append(
                f"Invalid date format: {value}. Expected YYYY-MM-DD"
            )

    elif field_type == "number" or any(
        word in field_name.lower() for word in ["count", "amount", "score", "rating"]
    ):
        # Validate numeric value
        if isinstance(value, (int, float)):
            print(f"   ✅ Valid numeric value: {value}")
        elif isinstance(value, str):
            try:
                float(value)
                print(f"   ✅ Valid numeric string: {value}")
            except ValueError:
                validation_errors.append(
                    f"Numeric field requires number or numeric string, got '{value}'"
                )
        else:
            validation_errors.append(
                f"Numeric field requires number, got {type(value).__name__}"
            )

    elif field_type == "boolean" or any(
        word in field_name.lower() for word in ["is_", "active", "enabled", "opt"]
    ):
        # Validate boolean value
        valid_boolean_values = [
            True,
            False,
            "true",
            "false",
            "yes",
            "no",
            "1",
            "0",
            1,
            0,
        ]
        if value in valid_boolean_values:
            print(f"   ✅ Valid boolean value: {value}")
        else:
            warnings.append(
                f"Boolean field value '{value}' may not be recognized. Consider: true/false, yes/no, 1/0"
            )

    # Step 3: Value length/size validation
    if isinstance(value, str) and len(value) > 1000:
        warnings.append(
            f"Very long text value ({len(value)} characters) may exceed field limits"
        )

    if isinstance(value, list) and len(value) > 50:
        warnings.append(f"Very long list ({len(value)} items) may exceed field limits")

    # Step 4: Display validation results
    if validation_errors:
        print(f"   ❌ Validation Errors:")
        for error in validation_errors:
            print(f"      • {error}")
        return None

    if warnings:
        print(f"   ⚠️  Validation Warnings:")
        for warning in warnings:
            print(f"      • {warning}")

    if not validation_errors and not warnings:
        print(f"   ✅ All validations passed")

    # Step 5: Prepare and show update code
    update_data = {field_name: value}

    print(f"\n💡 Validated update code:")
    print(f"```python")
    print(f"# Validated custom field update")
    print(f"update_data = {{")
    if isinstance(value, str):
        print(f"    '{field_name}': '{value}'")
    else:
        print(f"    '{field_name}': {value}")
    print(f"}}")
    print(f"")
    print(f"try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"    result = client.accounts.update({account_id}, update_data)")
    print(f"    print('✅ Custom field updated successfully with validation')")
    print(f"except Exception as e:")
    print(f"    print(f'❌ Error updating custom field: {{str(e)}}')")
    print(f"```")

    # UNCOMMENT TO ACTUALLY UPDATE:
    # if not validation_errors:  # Only update if no validation errors
    #     try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #         result = client.accounts.update(account_id, update_data)
    #         print(f"✅ Custom field '{field_name}' updated successfully with validation")
    #         return result
    #     except Exception as e:
    #         print(f"❌ Error updating custom field: {str(e)}")
    #         return None

    print(f"\n⚠️  Update code is commented out for safety. Uncomment to execute.")
    return None


# Example validation scenarios
validation_examples = [
    {
        "field_name": "Custom - Last Contact Date",
        "value": "2024-01-15",
        "field_type": "date",
        "description": "Valid date field",
    },
    {
        "field_name": "Custom - Donation Count",
        "value": "not_a_number",
        "field_type": "number",
        "description": "Invalid numeric field (should fail)",
    },
    {
        "field_name": "Custom - Is VIP",
        "value": "maybe",
        "field_type": "boolean",
        "description": "Questionable boolean value (should warn)",
    },
    {
        "field_name": "123",
        "value": "Valid text for numeric field name",
        "field_type": None,
        "description": "Numeric field name with text value",
    },
]

print("\n🔍 CUSTOM FIELD VALIDATION EXAMPLES:")
print("=" * 40)

for i, example in enumerate(validation_examples, 1):
    print(f"\n{i}. {example['description']}:")

    if test_account_id:
        validate_and_update_custom_field(
            client,
            test_account_id,
            example["field_name"],
            example["value"],
            example["field_type"],
        )
    else:
        print(f"   Field: {example['field_name']}")
        print(f"   Value: {example['value']}")
        print(f"   Expected Type: {example['field_type']}")
        print(f"   📝 No test account available - showing structure only")

In [ ]:
# 5.2: Error handling and recovery patterns
print("\n🔍 SECTION 5.2: Error Handling and Recovery Patterns")
print("=" * 55)


def robust_custom_field_update(client, account_id: str, field_updates: Dict[str, Any]):
    """Robust custom field update with comprehensive error handling."""

    print(
        f"\n🛡️  Robust update of {len(field_updates)} custom fields with error handling"
    )

    results = {
        "successful_updates": [],
        "failed_updates": [],
        "warnings": [],
        "total_attempted": len(field_updates),
    }

    print(f"\n💡 Robust update pattern:")
    print(f"```python")
    print(f"def robust_custom_field_update(client, account_id, field_updates):")
    print(f"    results = {{'successful': [], 'failed': [], 'warnings': []}}")
    print(f"    ")
    print(f"    # Strategy 1: Try updating all fields at once")
    print(f"    try:")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"        result = client.accounts.update(account_id, field_updates)")
    print(f"        results['successful'].extend(field_updates.keys())")
    print(f"        return results")
    print(f"    except Exception as e:")
    print(f"        print(f'Bulk update failed: {{str(e)}}')")
    print(f"        print('Falling back to individual field updates...')")
    print(f"    ")
    print(f"    # Strategy 2: Update fields individually")
    print(f"    for field_name, value in field_updates.items():")
    print(f"        try:")
    print(f"            single_update = {{field_name: value}}")
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # print(f"            result = client.accounts.update(account_id, single_update)")
    print(f"            results['successful'].append(field_name)")
    print(f"        except NeonValidationError as ve:")
    print(
        f"            results['failed'].append({{'field': field_name, 'error': 'validation', 'details': str(ve)}})"
    )
    print(f"        except NeonAPIError as ae:")
    print(
        f"            results['failed'].append({{'field': field_name, 'error': 'api', 'details': str(ae)}})"
    )
    print(f"        except Exception as e:")
    print(
        f"            results['failed'].append({{'field': field_name, 'error': 'unknown', 'details': str(e)}})"
    )
    print(f"    ")
    print(f"    return results")
    print(f"```")

    # Simulate the robust update process
    print(f"\n🔄 Simulating robust update process:")

    # Strategy 1: Try bulk update first
    print(
        f"\n1️⃣  Strategy 1: Attempting bulk update of all {len(field_updates)} fields..."
    )

    # UNCOMMENT TO TRY BULK UPDATE:
    # try:
    # SAFETY: Commented out to prevent database modification
    # SAFETY: Commented out to prevent database modification
    # # #     result = client.accounts.update(account_id, field_updates)
    #     results['successful_updates'] = list(field_updates.keys())
    #     print(f"   ✅ Bulk update successful! All {len(field_updates)} fields updated.")
    #     return results
    # except Exception as e:
    #     print(f"   ❌ Bulk update failed: {str(e)}")
    #     print(f"   🔄 Falling back to individual field updates...")

    print(f"   ⚠️  Bulk update simulation (commented out for safety)")
    print(f"   🔄 Proceeding to individual field update simulation...")

    # Strategy 2: Individual field updates
    print(f"\n2️⃣  Strategy 2: Updating fields individually...")

    for i, (field_name, value) in enumerate(field_updates.items(), 1):
        print(f"\n   Field {i}/{len(field_updates)}: '{field_name}'")

        # Simulate validation
        has_validation_issues = False

        # Check for obvious validation issues
        if "date" in field_name.lower() and not isinstance(value, str):
            has_validation_issues = True
            error_details = f"Date field requires string, got {type(value).__name__}"
        elif field_name == "" or value is None:
            has_validation_issues = True
            error_details = "Empty field name or null value"

        if has_validation_issues:
            results["failed_updates"].append(
                {"field": field_name, "error": "validation", "details": error_details}
            )
            print(f"      ❌ Validation failed: {error_details}")
        else:
            # UNCOMMENT TO ACTUALLY UPDATE:
            # try:
            #     single_update = {field_name: value}
            # SAFETY: Commented out to prevent database modification
            # SAFETY: Commented out to prevent database modification
            # # #     result = client.accounts.update(account_id, single_update)
            #     results['successful_updates'].append(field_name)
            #     print(f"      ✅ Updated successfully")
            # except NeonValidationError as ve:
            #     results['failed_updates'].append({
            #         'field': field_name,
            #         'error': 'validation',
            #         'details': str(ve)
            #     })
            #     print(f"      ❌ Validation error: {str(ve)}")
            # except Exception as e:
            #     results['failed_updates'].append({
            #         'field': field_name,
            #         'error': 'unknown',
            #         'details': str(e)
            #     })
            #     print(f"      ❌ Update error: {str(e)}")

            # Simulation
            results["successful_updates"].append(field_name)
            print(f"      ✅ Update simulation successful")

    return results


# Example robust update scenario
robust_update_example = {
    "Custom - Contact Method": "Email 1",
    "Custom - Is Active": True,
    "Custom - Last Update": datetime.now().strftime("%Y-%m-%d"),
    "Custom - Score": 85,
    "Custom - Invalid Date": 123,  # This should fail validation
    "Custom - Preferences": {"email": True, "phone": False},
}

print("\n🛡️  ROBUST ERROR HANDLING EXAMPLE:")
print("=" * 40)

if test_account_id:
    update_results = robust_custom_field_update(
        client, test_account_id, robust_update_example
    )

    # Display results summary
    print(f"\n📊 UPDATE RESULTS SUMMARY:")
    print(f"   Total attempted: {update_results['total_attempted']}")
    print(f"   Successful: {len(update_results['successful_updates'])}")
    print(f"   Failed: {len(update_results['failed_updates'])}")
    print(
        f"   Success rate: {len(update_results['successful_updates']) / update_results['total_attempted'] * 100:.1f}%"
    )

    if update_results["successful_updates"]:
        print(f"\n   ✅ Successful updates:")
        for field in update_results["successful_updates"]:
            print(f"      • {field}")

    if update_results["failed_updates"]:
        print(f"\n   ❌ Failed updates:")
        for failure in update_results["failed_updates"]:
            print(
                f"      • {failure['field']}: {failure['error']} - {failure['details']}"
            )
else:
    print("   📝 No test account available - showing error handling pattern only")
    for field_name, value in robust_update_example.items():
        print(f"      {field_name}: {value} ({type(value).__name__})")

## Section 6: Summary and Best Practices

Complete summary of custom field update patterns and best practices.

In [ ]:
# 6.1: Complete summary and best practices
print("📋 CUSTOM FIELD UPDATES - COMPLETE SUMMARY")
print("=" * 55)

print(f"\n🎯 SUMMARY OF DATA TYPES COVERED:")
print("=" * 35)

data_type_summary = [
    {
        "type": "Text/String",
        "examples": ["Simple text", "Long descriptions", "Special characters"],
        "format": "string",
        "notes": "Most common type, supports any text content",
    },
    {
        "type": "Numeric",
        "examples": ["Integers", "Floats", "Numeric strings"],
        "format": "number | string",
        "notes": "Can be passed as number or string representation",
    },
    {
        "type": "Boolean",
        "examples": ["true/false", "yes/no", "1/0"],
        "format": "boolean | string",
        "notes": "Multiple formats accepted, consider API preferences",
    },
    {
        "type": "date",
        "examples": ["YYYY-MM-DD format", "ISO dates"],
        "format": "string (YYYY-MM-DD)",
        "notes": "Use consistent date format, validate before sending",
    },
    {
        "type": "List/Array",
        "examples": ["Python lists", "JSON arrays", "Comma-separated"],
        "format": "list | string",
        "notes": "Multiple representation options, test which works best",
    },
    {
        "type": "Complex/Object",
        "examples": ["Nested objects", "JSON strings"],
        "format": "dict | string",
        "notes": "Use for structured data, may need JSON serialization",
    },
]

for i, data_type in enumerate(data_type_summary, 1):
    print(f"\n{i}. {data_type['type']}:")
    print(f"   Format: {data_type['format']}")
    print(f"   Examples: {', '.join(data_type['examples'])}")
    print(f"   Notes: {data_type['notes']}")

print(f"\n💡 BEST PRACTICES FOR CUSTOM FIELD UPDATES:")
print("=" * 45)

best_practices = [
    "🔍 Discovery First",
    "   • Always discover available custom fields before attempting updates",
    "   • Use get_search_fields() and get_output_fields() to identify custom fields",
    "   • Cache discovery results to avoid repeated API calls",
    "",
    "✅ Validation Strategy",
    "   • Validate field names against known custom field patterns",
    "   • Validate data types before sending to API",
    "   • Use consistent date formats (YYYY-MM-DD recommended)",
    "   • Check value lengths and sizes for reasonable limits",
    "",
    "🛡️  Error Handling",
    "   • Implement try-catch blocks for all update operations",
    "   • Distinguish between validation errors and API errors",
    "   • Use fallback strategies (bulk → individual updates)",
    "   • Log failed updates for debugging and retry",
    "",
    "🎯 Data Type Guidelines",
    "   • Text: Use strings, escape special characters if needed",
    "   • Numbers: Can use numeric types or string representations",
    "   • Booleans: Test true/false vs yes/no vs 1/0 with your API",
    "   • Dates: Always use YYYY-MM-DD format strings",
    "   • Lists: Test list vs JSON string vs comma-separated formats",
    "   • Objects: Use dict or JSON string, test which works better",
    "",
    "⚡ Performance Optimization",
    "   • Batch multiple custom field updates in single operation when possible",
    "   • Cache field validation results",
    "   • Use appropriate timeout and retry mechanisms",
    "   • Monitor update performance and adjust strategies",
    "",
    "🔧 Development Practices",
    "   • Create reusable custom field update functions",
    "   • Implement comprehensive logging for debugging",
    "   • Build test suites for different data type scenarios",
    "   • Document organization-specific custom field mappings",
    "",
    "🚀 Advanced Patterns",
    "   • Implement custom field schema validation",
    "   • Build custom field discovery automation",
    "   • Create custom field synchronization between environments",
    "   • Develop custom field usage analytics",
]

for practice in best_practices:
    if practice:
        print(practice)

print(f"\n🎓 KEY IMPLEMENTATION INSIGHTS:")
print("=" * 35)

key_insights = [
    "🔢 Numeric field names (e.g., '123', '456') are commonly used for custom fields",
    "➖ Dash-separated names (e.g., 'Custom - Field Name') are well-supported patterns",
    "📊 Data type inference helps with validation but always test with actual API",
    "🔄 Multiple update strategies (bulk vs individual) provide resilience",
    "🛡️  Comprehensive error handling is essential for production reliability",
    "⚡ Performance varies with field count and data complexity",
    "🎯 Organization-specific patterns may differ from generic examples",
    "📝 Thorough testing with real data is crucial before production deployment",
]

for insight in key_insights:
    print(f"  {insight}")

print(f"\n⚠️  IMPORTANT REMINDERS:")
print("=" * 25)

reminders = [
    "🔒 All update examples in this notebook are commented out for safety",
    "🧪 Test custom field updates in a development environment first",
    "📋 Always backup data before performing bulk custom field updates",
    "🔍 Verify custom field updates by retrieving and checking the data",
    "📊 Monitor API rate limits when performing many custom field updates",
    "🎯 Custom field behavior may vary between different Neon CRM organizations",
    "💾 Consider data migration strategies for large-scale custom field changes",
    "🔄 Implement rollback procedures for critical custom field updates",
]

for reminder in reminders:
    print(f"  {reminder}")

print(f"\n✅ CUSTOM FIELD UPDATES TESTING COMPLETED")
print(f"\n📝 This comprehensive guide provides examples for updating custom fields")
print(f"   with all major data types. Use these patterns as a foundation for")
print(f"   building robust custom field management in your applications.")
print(f"\n🚀 Ready to implement custom field updates with confidence!")